# Configuração de ambiente

In [ ]:
%pip install crewai crewai-tools poetry vllm
!curl -fsSL https://ollama.com/install.sh | sh
!pip install pymupdf langchain-community langchain-core langgraph faiss-cpu
!pip install langchain --upgrade

In [2]:
!nohup ollama serve > ollama.log &
!nohup ollama run llava:7b &

nohup: redirecting stderr to stdout
nohup: appending output to 'nohup.out'


# Criação da RAG leve com base no PDF

In [ ]:
import pymupdf  # PyMuPDF
import os
import zipfile

# Caminho do arquivo ZIP enviado pelo usuário
zip_path = "RAG.zip"
extracted_path = "extracted_pdfs"

# Extraindo os PDFs
os.makedirs(extracted_path, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)

# Função para extrair texto de PDFs
def extract_text_from_pdfs(pdf_folder):
    text_data = ""
    for pdf_file in os.listdir(pdf_folder):
        if pdf_file.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, pdf_file)
            doc = pymupdf.open(pdf_path)
            for page in doc:
                text_data += page.get_text("text") + "\n\n"
    return text_data

# Extraindo texto dos PDFs
medical_texts = extract_text_from_pdfs(extracted_path)

# Salvando em um arquivo de texto
medical_texts_path = "medical_texts.txt"
with open(medical_texts_path, "w", encoding="utf-8") as f:
    f.write(medical_texts)

# Retornar o caminho do arquivo salvo
medical_texts_path


# Criando objeto do RAG

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain.schema import SystemMessage, HumanMessage
from langchain_community.vectorstores.faiss import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langgraph.graph import StateGraph, START, END
from pydantic import BaseModel, Field
import numpy as np


# Carregar embeddings do Sentence-Transformers
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Carregar e processar documentos médicos (já extraídos dos PDFs)
loader = TextLoader("medical_texts.txt")  # Arquivo consolidado com informações médicas extraídas
texts = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.split_documents(texts)

# Criar banco vetorial FAISS
vectorstore = FAISS.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever()

# Configurar modelo LLM
llm = ChatOllama(model="llava:7b")  # Trocar conforme necessário

# Leitura do modelo selecionado

In [ ]:
from Image_Segmentation.network import U_Net,R2U_Net,AttU_Net,R2AttU_Net
import torch
import torch.nn.functional as F
from torchvision import transforms as T
from PIL import Image
import random

model = U_Net(img_ch=3,output_ch=1)
model.load_state_dict(torch.load('./U_Net-100-0.0005-70-0.6667.pkl', map_location=torch.device('cpu')))

# Criação dos agentes e fluxo

In [64]:
# Configurar agentes
class SegmentationAgent:
  def __init__(self,model,threshold=.5,image_size=224,mode='train',augmentation_prob=0.4):
    self.model = model
    self.image_size = image_size
    self.mode = mode
    self.RotationDegree = [0,90,180,270]
    self.augmentation_prob = augmentation_prob

  def segment_image(self, state):
    """Reads an image from a file and preprocesses it and returns."""
    image = Image.open(state.image_path)

    aspect_ratio = image.size[1]/image.size[0]

    Transform = []

    ResizeRange = random.randint(300,320)
    Transform.append(T.Resize((int(ResizeRange*aspect_ratio),ResizeRange)))
    p_transform = random.random()

    # if (self.mode == 'train') and p_transform <= self.augmentation_prob:
    #   RotationDegree = random.randint(0,3)
    #   RotationDegree = self.RotationDegree[RotationDegree]
    #   if (RotationDegree == 90) or (RotationDegree == 270):
    #     aspect_ratio = 1/aspect_ratio

    #   Transform.append(T.RandomRotation((RotationDegree,RotationDegree)))

    #   RotationRange = random.randint(-10,10)
    #   Transform.append(T.RandomRotation((RotationRange,RotationRange)))
    #   CropRange = random.randint(250,270)
    #   Transform.append(T.CenterCrop((int(CropRange*aspect_ratio),CropRange)))
    #   Transform = T.Compose(Transform)

    #   image = Transform(image)

    #   ShiftRange_left = random.randint(0,20)
    #   ShiftRange_upper = random.randint(0,20)
    #   ShiftRange_right = image.size[0] - random.randint(0,20)
    #   ShiftRange_lower = image.size[1] - random.randint(0,20)
    #   image = image.crop(box=(ShiftRange_left,ShiftRange_upper,ShiftRange_right,ShiftRange_lower))

    #   if random.random() < 0.5:
    #     image = F.hflip(image)

    #   if random.random() < 0.5:
    #     image = F.vflip(image)

    #   Transform = T.ColorJitter(brightness=0.2,contrast=0.2,hue=0.02)

    #   image = Transform(image)

    #   Transform =[]


    Transform.append(T.Resize((int(256*aspect_ratio)-int(256*aspect_ratio)%16,256)))
    Transform.append(T.ToTensor())
    Transform = T.Compose(Transform)

    image = Transform(image)

    Norm_ = T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    image = Norm_(image)

    SR = model(image.unsqueeze(0))
    SR_s = F.sigmoid(SR)

    SR_image = (SR_s>.9)*image

    return {"image_data": SR_image.detach().numpy()}
#tensor_image = SegmentationAgent('/content/image/ISIC_0036209.jpg', model).segment_image()
#plt.imshow(tensor_image[0].permute(1, 2, 0))

class DiagnosticAgent:
    def __init__(self, llm):
        self.llm = llm

    def analyze_lesion(self, state):
        # Simulação da análise de imagem segmentada
        # content_str = """Utilize técnicas avançadas de análise de imagem para identificar padrões visuais e
        # características relevantes em lesões cutâneas, permitindo uma avaliação inicial detalhada para auxiliar em
        # investigações médicas."""
        # content_str = """Com base na imagem fornecida, descreva detalhadamente as características visuais específicas
        # da lesão analisada. Inclua informações sobre textura, cor, bordas, forma e padrões internos observados. Compare
        # essas características com padrões conhecidos de lesões cutâneas documentadas na literatura médica. Apresente um
        # parecer objetivo, sem fornecer diagnósticos, apenas detalhando as semelhanças e diferenças com condições
        # dermatológicas reconhecidas."""
        content_str = """Descreva a imagem com base no checklist de 7 pontos."""
        msg = [{"role": "user", "content": content_str, "images": [state.image_data]}]
        return {"diagnosis": self.llm.invoke(msg).content}#"Lesão apresenta pigmentação irregular e bordas assimétricas. Possível risco moderado."}

class PrognosticAgent:
    def __init__(self, llm):
        self.llm = llm

    def assess_risk(self, state):
      # content_str = """Analise as imagens das lesões para identificar tendências visuais e padrões associados à
      # progressão da condição, colaborando com a equipe para fornecer uma avaliação detalhada da possível evolução do
      # quadro.
      # """
      # content_str="""Com base na imagem analisada, descreva possíveis padrões evolutivos da lesão observada. Considere
      # fatores como variações na coloração, bordas, simetria e possíveis mudanças estruturais ao longo do tempo. Compare
      # essas características com padrões documentados na literatura médica sobre a progressão de condições
      # dermatológicas. Relate quais indícios visuais podem sugerir maior ou menor risco de agravamento.
      # """
      content_str="""Com base na imagem disponibilizada, descreva possíveis padrões evolutivos da lesão observada. Considere
      fatores como variações na coloração, bordas, simetria e possíveis mudanças estruturais ao longo do tempo. Compare
      essas características com padrões documentados na literatura médica sobre a progressão de condições
      dermatológicas. Relate quais indícios visuais podem sugerir maior ou menor risco de agravamento.
      """
      msg = [{"role": "user", "content": content_str, "images": [state.image_data]}]
      return {"prognosis": self.llm.invoke(msg).content}

class SummaryAgent:
    def __init__(self, llm):
        self.llm = llm

    def summarize(self, state):
      content_str=f"""Com base nas descrições fornecidas pelos especialistas, compile um relatório estruturado sobre a lesão analisada. O relatório deve incluir:
1️⃣ Características Visuais: Resumo das observações sobre textura, coloração, bordas e padrões internos da lesão.
2️⃣ Análise Comparativa: Como essas características se relacionam com padrões conhecidos de lesões dermatológicas.
3️⃣ Estimativa de Evolução: Possíveis mudanças na lesão ao longo do tempo, considerando padrões documentados.
4️⃣ Recomendações: Medidas sugeridas para acompanhamento, investigações adicionais ou intervenção clínica, se aplicável.

Apresente o relatório em formato JSON, garantindo clareza e organização das informações.
      [[{state.diagnosis}]]
      <<{state.prognosis}>>
      """
      msg = [{"role": "user", "content": content_str, "images": [state.image_data]}]
      return {"validation": self.llm.invoke(msg).content}

# Configurar agente crítico
class CriticalReviewAgent:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm

    def validate_diagnosis(self, state):
        retrieved_docs = self.retriever.invoke(state.diagnosis)
        return {"final_report": f"Confirmação baseada em literatura médica: {retrieved_docs[0].page_content[:200]}..."}


# Definir o esquema de estado inicial
class GraphState(BaseModel):
    image_path: str
    image_data: np.ndarray = Field(default_factory=lambda: np.zeros(10))
    diagnosis: str = None
    prognosis: str = None
    validation: str = None
    final_report: str = None
    class Config:
        arbitrary_types_allowed = True

# Criar fluxo no LangGraph
graph = StateGraph(GraphState)

graph.add_node("segmentation", SegmentationAgent(model).segment_image)
graph.add_node("diagnostic", DiagnosticAgent(llm).analyze_lesion)
graph.add_node("prognostic", PrognosticAgent(llm).assess_risk)
graph.add_node("summary", SummaryAgent(llm).summarize)
graph.add_node("critical_review", CriticalReviewAgent(retriever, llm).validate_diagnosis)

# Definir conexões do fluxo
graph.add_edge(START, "segmentation")
graph.add_edge("segmentation", "diagnostic")
graph.add_edge("diagnostic", "prognostic")
graph.add_edge("prognostic", "summary")
graph.add_edge("summary", "critical_review")
graph.add_edge("critical_review", END)

g = graph.compile()

# Executar o fluxo de teste
output = g.stream({"image_path": "/content/image/ISIC_0036252.jpg"}, stream_mode="values")
print(output)


<generator object Pregel.stream at 0x7cfdbdb80fc0>


In [65]:
# output = g.stream({"image_data": "./ISIC_0036252.jpg"}, stream_mode="values")
# # print(output)

for event in output:
  print(event)

{'image_path': '/content/image/ISIC_0036252.jpg'}
{'image_path': '/content/image/ISIC_0036252.jpg', 'image_data': array([[[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ...,
         [-0., -0., -0., ..., -0., -0., -0.],
         [-0., -0., -0., ..., -0., -0., -0.],
         [-0., -0., -0., ..., -0., -0., -0.]],

        [[-0.,  0.,  0., ..., -0., -0., -0.],
         [ 0.,  0.,  0., ...,  0., -0., -0.],
         [ 0.,  0.,  0., ...,  0., -0., -0.],
         ...,
         [-0., -0., -0., ..., -0., -0., -0.],
         [-0., -0., -0., ..., -0., -0., -0.],
         [-0., -0., -0., ..., -0., -0., -0.]],

        [[-0., -0., -0., ..., -0., -0., -0.],
         [-0., -0., -0., ..., -0., -0., -0.],
         [-0., -0., -0., ..., -0., -0., -0.],
         ...,
         [-0., -0., -0., ..., -0., -0., -0.],
         [-0., -0., -0., ..., -0., -0., -0.],
         [-0., -0., -0., ..., -0., -0., -0.]]]], dtype=flo

In [66]:
print("Diagnóstico:\n", event['diagnosis'],'\n', 100*"=",'\n')
print("Prognóstico:\n", event['prognosis'],'\n', 100*"=",'\n')
print("Relatório:\n", event['validation'])

Diagnóstico:
  A imagem parece ser uma foto que mostra um ambiente interno, provavelmente dentro de uma casa ou apartamento. O ponto 1 do checklist sugere que houve alterações no espaço, mas não é possível identificar exatamente qual a mudança foi sem mais contexto.

O ponto 2 fala de uma superfície branca, mas nenhum elemento na imagem seja identificado como tendo essa cor.

O ponto 3 diz que há cores vivas, mas não é possível dizer quais são elas sem mais informações.

O ponto 4 sugestiona a existência de um ambiente feito para o foco, e isso parece ser o caso na imagem. Há uma luz bem distribuída no espaço, com iluminação da luz principal no teto e possivelmente outras fontes de luz suplementares como lâmpadas ou iluminadores.

O ponto 5 menciona a presença de elementos naturais, mas na imagem não há nenhuma evidência de materialidade ou textura natural.

O ponto 6 diz que houve uma mudança no espaço, mas como mencionado anteriormente, sem mais detalhes é impossível dizer quê foi a 

In [63]:
import base64

# Caminho da imagem
image_path = "./A-Cat.jpg"

# Converter a imagem para Base64
with open(image_path, "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode("utf-8")

msg = [{"role": "user",
        "content": "Descreva a imagem",
        # "content": "Descreva a imagem com base no checklist de 7 pontos para detecção de cancer de pele .",
        "images": [encoded_string]
        }]
response = llm.invoke(msg)
print(response.content)

 A imagem mostra uma cena de um ambiente interno, que parece ser uma sala ou corredor. Há um pequeno objeto roxo na parte superior esquerda da foto, o que pode ser uma estante ou prateleira. Atrás disso, há uma janela com um vidro e uma trave.

O peixe está nadando em uma piscina dentro de um ambiente interno, talvez uma sala ou corredor, e parece estar vendo para a frente da cena. Ele tem uma barba curta e ombreiro mais escuro do que a parte superior do seu corpo. O peixe está em um estado de relaxamento, com as finas aletas de sua nadade ao ar livre.

A imagem é de natureza artificial e parece estar sendo utilizada para ilustrar uma situação ou ser parte de um projeto artístico ou de um artigo informativo relacionado a peixes. O estilo da foto é casuoso e naturalista, sem nenhum tipo de edição ou filtro que desfigure o aspecto do peixe ou do ambiente. 
